<a href="https://colab.research.google.com/github/jdowner212/cs577_addernet/blob/main/AdderNet_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
import skimage
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# so we can run original addernet:
from torch.torch_version import TorchVersion
import numpy as np
import torch

In [224]:
hard_tanh(3)

1

In [225]:
'''
Functions described in paper
'''

def L1(a,b):
    return -1*np.abs(a-b)

def hard_tanh(value):
    if value > 1:
        return 1
    elif value < -1:
        return -1
    else:
        return value

'''Equation 1'''
# modified by Equation 2

'''Equation 5'''
def dY_dF_element(image,filters,m,n,i,j,k,t):
    return image[k,m+i,n+j] - filter[t,k,i,j]

'''Equation 6''' # clipped, full-precision gradient
def dY_dImage_element(image,filters,m,n,i,j,k,t):
    return hard_tanh(filter[t,k,i,j] - image[k,m+i,n+j])

def window_gradients(image,filters,m,n,t):
    dy_df_window = np.zeros_like(filters)
    dy_dx_window = np.zeros_like(image)

    num_filters, k_depth, k_height, k_width = filters.shape
    for k in range(k_depth):
        for j in range(k_width):
            for i in range(k_height):
                img_minus_f = dY_dF_element(image,filters,m,n,i,j,k,t)
                dy_df_window[t,k,i,j] = img_minus_f
                dy_dx_window[k,i,j]   = hard_tanh(img_minus_f)
                
    return dy_df_window, dy_dx_window

'''Equation 2'''
def Y_adder(image, F, m, n, t, similarity_f=L1): # image, group fo filters, row#, col#, filter#, similarity function
    sum_ = 0
    num_filters, k_depth, k_height, k_width = F.shape
    for k in range(k_depth):
        for j in range(k_width):
            for i in range(k_height):
                sum_ += similarity_f(image[k, m+i, n+j], F[t,k,i,j])
    return sum_

In [247]:
images = np.random.randint(0,10,(1,3,5,5))
filters = np.random.randint(0,10,(5,3,3,3))
forward_batch(images,filters,1,0)#.shape


array([[[[[ -85.,  -95.,  -71.],
          [-107.,  -90., -103.],
          [-109.,  -57., -101.]]],


        [[[ -96.,  -82.,  -90.],
          [ -88.,  -77.,  -82.],
          [ -94.,  -94.,  -86.]]],


        [[[ -92.,  -82.,  -96.],
          [ -90.,  -85.,  -86.],
          [-100.,  -82.,  -78.]]],


        [[[-105.,  -83.,  -95.],
          [ -93.,  -76., -111.],
          [ -95.,  -97., -101.]]],


        [[[ -94.,  -96., -100.],
          [ -74.,  -87.,  -84.],
          [ -88.,  -72.,  -84.]]]]])

In [256]:
model = tf.keras.models.Sequential()

In [ ]:
model.add.layers(tf.keras.layers.Conv2D(filters))

In [260]:
tf.keras.layers.Conv2D

keras.layers.convolutional.conv2d.Conv2D

In [ ]:

'''Equation 3'''
# ignore -- CNN formula

'''Equation 4'''
# ignore -- updated with equation 5

'''Equation 7'''
# ignore -- hard_tanh implemented previously

'''Equation 8'''
# ignore -- CNN formula

'''Equation 9'''
# def var_Y_adder(X,F,variance_f=torch.var):
    # check torch.var documentation: https://pytorch.org/docs/stable/generated/torch.var.html
    # not sure if we can call torch.var(X) with default parameters
    # or if we need to specify. Does this output a scalar or a tensor?
# Trying K.var as tensorflow substitute for torch.var -- make sure they work the same
# or tf.var?
def var_Y_adder(X,F,variance_f=K.var):
    var_X = variance_f(X)
    var_F = variance_f(F)
    ###
    _, c_in, d, _ = F.shape
    pi = np.pi

    return np.sqrt(pi/2)*(d**2)*(c_in)*(var_X + var_F)

'''Equation 10'''
def batch_norm(minibatch, gamma, beta):
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    gamma*(minibatch-mean)/std + beta
    return gamma*(minibatch-mean)/std + beta

'''Equation 11'''
def dL_dMinibatch_i(minibatch,dL_dy,i,L,gamma):
    # In dL_dy, y is the result of applying batch_norm to the minibatch
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    
    sum_ = 0
    for j in range(m):
        x_term = (minibatch[i]-minibatch[j])*(minibatch[j]-mean)/std
        sum_ += (dL_dy[i] - dL_dy[j]*(1 + x_term))
    sum_ *= gamma/((m**2)*std)
    
    return sum_

'''Equation 12'''
# update rule for F
def delta_F_l(adaptive_lr_l, dL_dF_l, gamma):
    # the update delta for the filter in layer l
    return gamma*adaptive_lr_l*dL_dF_l

'''Equation 13'''
def adaptive_lr_l(dL_dF_l, eta, k):
    # k = number of elements in F_l -- I think equal to len(dL_dF_1)
    # in which case we don't need to explicitly provide it
    
    
    # l2_norm = torch.sqrt([g**2 for g in dL_dF_l])
    l2_norm = K.sqrt([g**2 for g in dL_dF_l]) # make sure torch.sqrt and K.sqrt are equivalents
    
    
    return eta*np.sqrt(k)/l2_norm

In [ ]:
def backward(X,F,Y_hat,Y,L):
    print('\n\n**Backward**\n\n')
    X     = self.X
    Y     = self.Y
    Y_hat = self.Y_hat
    W_L1  = self.W_L1
    W_L2  = self.W_L2
    W_L3  = self.W_L3

    # d_loss
    d_yhat = self.loss.backward()
    print('d_yhat.shape:       ', d_yhat.shape)

    # layer 3
    d_ZW3_plus_b   = self.softmax.backward(d_yhat)
    [d_ZW3, d_b3]  = self.add__L3.backward(d_ZW3_plus_b)
    [d_Z2,  d_W3]  = self.mult_L3.backward(d_ZW3)
    print('d_ZW3_plus_b.shape: ',self.d_Z3_plus_b.shape)
    print('d_W3.shape:         ',self.d_W3.shape)
    print('d_Z2.shape:         ',self.d_Z2.shape)

    # layer 2
    d_ZW2_plus_b   = self.sig__L2.backward(d_Z2)
    [d_ZW2, d_b2]  = self.add__L2.backward(d_ZW2_plus_b)
    [d_Z1,   d_W2] = self.mult_L2.backward(d_ZW2)
    print('d_ZW2_plus_b.shape: ',self.d_ZW2_plus_b.shape)
    print('d_W2.shape:         ',self.dW2.shape)
    print('d_Z1.shape:         ',self.d_Z1.shape)

    # layer 1
    d_ZW1_plus_b  = self.sig__L1.backward(d_Z1)
    [d_XW1, d_b1] = self.add__L1.backward(d_ZW1_plus_b)
    [d_X,   d_W1] = self.mult_L1.backward(d_XW1)
    print('d_ZW1_plus_b.shape: ',self.d_sig_Z1.shape)
    print('d_W1.shape:         ',self.d_W1.shape)
    print('d_X.shape:          ',self.d_X.shape)

    return d_W1, d_b1, d_W2, d_b2, d_W3, d_b3

In [109]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Function
import math

def adder2d_function(X, W, stride=1, padding=0):
    n_filters, d_filter, h_filter, w_filter = W.shape
    n_x, d_x, h_x, w_x = X.size()

    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    h_out, w_out = int(h_out), int(w_out)
    X_col = torch.nn.functional.unfold(X.view(1, -1, h_x, w_x), h_filter, dilation=1, padding=padding, stride=stride).view(n_x, -1, h_out*w_out)
    X_col = X_col.permute(1,2,0).contiguous().view(X_col.size(1),-1)
    W_col = W.view(n_filters, -1)
    
    out = adder.apply(W_col,X_col)
    
    out = out.view(n_filters, h_out, w_out, n_x)
    out = out.permute(3, 0, 1, 2).contiguous()
    
    return out

class adder(Function):
    @staticmethod
    def forward(ctx, W_col, X_col):
        ctx.save_for_backward(W_col,X_col)
        display(X_col)
        display(W_col)
        output = -(W_col.unsqueeze(2)-X_col.unsqueeze(0)).abs().sum(1)
        return output

    @staticmethod
    def backward(ctx,grad_output):
        W_col,X_col = ctx.saved_tensors
        grad_W_col = ((X_col.unsqueeze(0)-W_col.unsqueeze(2))*grad_output.unsqueeze(1)).sum(2)
        grad_W_col = grad_W_col/grad_W_col.norm(p=2).clamp(min=1e-12)*math.sqrt(W_col.size(1)*W_col.size(0))/5
        grad_X_col = (-(X_col.unsqueeze(0)-W_col.unsqueeze(2)).clamp(-1,1)*grad_output.unsqueeze(1)).sum(0)
        
        return grad_W_col, grad_X_col
    
class adder2d(nn.Module):

    def __init__(self,input_channel,output_channel,kernel_size, stride=1, padding=0, bias = False):
        super(adder2d, self).__init__()
        self.stride = stride
        self.padding = padding
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.kernel_size = kernel_size
        self.adder = torch.nn.Parameter(nn.init.normal_(torch.randn(output_channel,input_channel,kernel_size,kernel_size)))
        self.bias = bias
        if bias:
            self.b = torch.nn.Parameter(nn.init.uniform_(torch.zeros(output_channel)))

    def forward(self, x):
        output = adder2d_function(x,self.adder, self.stride, self.padding)
        if self.bias:
            output += self.b.unsqueeze(0).unsqueeze(2).unsqueeze(3)
        
        return output

In [278]:
# dimensions: NxHxWxC

def addernet_single_step(window, filter, similarity_f=L1, b=None):
    """
    window -- k_depth x k_height x k_width
    filter -- k_depth x k_height x k_width
    b      -- 1x1x1
    Z      -- scalar
    """
    if not b:
        b = np.zeros((1,1,1))

    k_height, k_width, k_depth = filter.shape

    Z=0
    for k in range(k_depth):
        for j in range(k_width):
            for i in range(k_height):
                out += similarity_f(window[k, i, j], filter[k,i,j])

    Z += b.astype(float)

    return Z


def addernet_forward(X, F, B, s=1, p=0):
    """    
    X -- n_tensors x H x W x c_in
    F -- k x k x k x c_out
    b -- n_filters x 1 x 1 x 1
    Z -- n_tensors x H_new x W_new, c_out
    cache -- info needed for backward pass
    """
    n_tensors, H, W, c_in = X.shape
    k, k, k, n_filters    = F.shape

    H_new = int((H + 2*p - k)/s)+1
    W_new = int((W + 2*p - k)/s)+1
    c_out = int((c_in + 2*p - k)/s)+1
    
    Z = np.zeros([n_tensors, H_new, W_new, c_out])
    X_padded = np.pad(X, ((0,0), (p,p), (p,p), (0,0)), 'constant', constant_values = (0,0))
    
    for i in range(n_tensors):            # loop over batches               
        this_x = X_padded[i,:,:,:]        # select ith image in batch
        for h in range(H_new):            # traverse height
            for w in range(W_new):        # traverse width
                for c in range(c_out):    # traverse depth/filters
                    
                    v0 = h*s
                    v1 = h*s + k
                    h0 = w*s 
                    h1 = w*s + k
                    
                    this_window = this_x[v0:v1,h0:h1,:]

                    Z[i, h, w, c] = addernet_single_step(this_window, F[:, :, :, c], B[:,:,:,c])
                                        
    assert(Z.shape == (n_tensors, H_new, W_new, c_out))
    
    cache = (X, W, B, s, p)
    
    return Z, cache

def addernet_backward(dZ_upstream, cache):
    """
    dZ_upstream (dL/dZ) -- n_tensors x H_up x W_up x c_up
    cache (values from previous layers) -- (X, W, B, s, p)               
    
    Output:
    dX -- dL/dX, shape n_tensors x H_down x W_down x c_down
    dW -- dL/dW, shape k x k x k x n_filters
    dB -- dL/dB, shape 1 x 1 x 1 x c_up
    """
    
    X, W, B, s, p = cache
    n_tensors, H_down, W_down, c_down = X.shape
    k, k, k, n_filters = W.shape
    
    n_tensors, H_up, W_up, c_up = dZ_upstream.shape
    

    dX_down = np.zeros((n_tensors, H_down, W_down, c_down))                           
    dW = np.zeros((k, k, k, n_filters))
    dB = np.zeros((1, 1, 1, n_filters))

    X_padded = np.pad(X, ((0,0), (p,p), (p,p), (0,0)), 'constant', constant_values = (0,0))
    dX_down_padded = np.pad(dX_down, ((0,0), (p,p), (p,p), (0,0)), 'constant', constant_values = (0,0))
    
    for i in range(n_tensors):                       
        x = X_padded[i]
        dx = dX_down_padded[i]
        
        for h in range(H_up):                   # loop over vertical axis of the output volume
            for w in range(W_up):               # loop over horizontal axis of the output volume
                for c in range(c_up):           # loop over the channels of the output volume
                    
                    v0 = h
                    v1 = v0 + k
                    h0 = w
                    h1 = h0 + k
                    
                    window = x[v0:v1, h0:v1, :]
                    dx[v0:v1, v0:v1, :] += W[:,:,:,c] * dZ_upstream[i, h, w, c] #

                    dW[:,:,:,c] += window * dZ_upstream[i, h, w, c]
                    dB[:,:,:,c] += dZ_upstream[i, h, w, c]
                    
        dX_down[i, :, :, :] = dx[p:-p, p:-p, :]
    
    assert(dX_down.shape == (n_tensors, H_down, W_down, c_down))
    
    return dX_down, dW, dB